In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)

from itertools import product
from sklearn.preprocessing import LabelEncoder

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import time
import sys
import gc
import pickle
sys.version_info

sys.version_info(major=3, minor=7, micro=6, releaselevel='final', serial=0)

In [2]:
from module.data.read_data import *

In [3]:
train = sales_file_processing()

In [4]:
test = test_file_processing().set_index('ID')

items = pd.read_csv(ITEMS_FILE)
shops = shops_file_processing()
cats = categories_file_processing()

In [5]:
train = train[train.item_price<100000]
train = train[train.item_cnt_day<1001]

In [6]:
median = train[(train.shop_id==32)&(train.item_id==2973)&(train.date_block_num==4)&(train.item_price>0)].item_price.median()
train.loc[train.item_price<0, 'item_price'] = median

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2935847 entries, 0 to 2935848
Data columns (total 6 columns):
date              object
date_block_num    int64
shop_id           int64
item_id           int64
item_price        float64
item_cnt_day      float64
dtypes: float64(2), int64(3), object(1)
memory usage: 156.8+ MB


In [8]:
ts = time.time()
cols = ['date_block_num','shop_id','item_id']
group = train.groupby(cols).agg({'item_cnt_day': ['sum']})

new_columns = ['item_cnt_month']
group.columns = new_columns
group.reset_index(inplace=True)
group[new_columns] = group[new_columns].astype(np.float16)

time.time() - ts

1.6565687656402588

In [9]:
ts = time.time()
cols = ['date_block_num','shop_id','item_id']
group = train.groupby(cols).agg({'item_cnt_day': ['sum'], 
                                 'item_price': ['first', 'last', 'min', 'mean', 'max']})

new_columns = ['item_cnt_month', 'item_price_open', 'item_price_close', 'item_price_min', 'item_price_mean', 'item_price_max']
group.columns = new_columns
group.reset_index(inplace=True)
group[new_columns] = group[new_columns].astype(np.float16)

time.time() - ts

3.962399482727051

In [10]:
group.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1608271 entries, 0 to 1608270
Data columns (total 9 columns):
date_block_num      1608271 non-null int64
shop_id             1608271 non-null int64
item_id             1608271 non-null int64
item_cnt_month      1608271 non-null float16
item_price_open     1608271 non-null float16
item_price_close    1608271 non-null float16
item_price_min      1608271 non-null float16
item_price_mean     1608271 non-null float16
item_price_max      1608271 non-null float16
dtypes: float16(6), int64(3)
memory usage: 55.2 MB


In [11]:
item_ids = set(train['item_id'].unique())
# item_ids.shape
item_ids_test = set(test['item_id'].unique())
# item_ids_test.shape
len(item_ids & item_ids_test)

4737

In [12]:
date_block_nums = [i for i in range(35)]
shop_ids = test['shop_id'].unique()
item_ids = test['item_id'].unique()
columns = ["date_block_num", "shop_id", "item_id"]

index = pd.MultiIndex.from_product([date_block_nums, shop_ids, item_ids], names=columns)
matrix = pd.DataFrame(index = index).reset_index()

In [13]:
matrix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7497000 entries, 0 to 7496999
Data columns (total 3 columns):
date_block_num    int64
shop_id           uint64
item_id           uint64
dtypes: int64(1), uint64(2)
memory usage: 171.6 MB


In [14]:
matrix['date_block_num'] = matrix['date_block_num'].astype(np.int8)
matrix['shop_id'] = matrix['shop_id'].astype(np.int8)
matrix['item_id'] = matrix['item_id'].astype(np.int16)
matrix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7497000 entries, 0 to 7496999
Data columns (total 3 columns):
date_block_num    int8
shop_id           int8
item_id           int16
dtypes: int16(1), int8(2)
memory usage: 28.6 MB


In [15]:
matrix.sort_values(cols,inplace=True)

In [16]:
matrix.head()

,date_block_num,shop_id,item_id
22987,0,2,30
20994,0,2,31
20995,0,2,32
22492,0,2,33
22491,0,2,38


In [17]:
matrix = pd.merge(matrix, group,  how='left', on=columns)

In [18]:
matrix = pd.merge(matrix, shops, on=['shop_id'], how='left')
matrix = pd.merge(matrix, items, on=['item_id'], how='left')
matrix = pd.merge(matrix, cats, on=['item_category_id'], how='left')

In [19]:
matrix['item_category_id'] = matrix['item_category_id'].astype(np.uint8)
matrix.drop(['item_name'], axis=1, inplace=True)

In [20]:
matrix.head()

,date_block_num,shop_id,item_id,item_cnt_month,item_price_open,item_price_close,item_price_min,item_price_mean,item_price_max,city_id,item_category_id,category_id,subcategory_id,subsubcategory_id
0,0,2,30,NaN,NaN,NaN,NaN,NaN,NaN,1,40,6,9,22
1,0,2,31,NaN,NaN,NaN,NaN,NaN,NaN,1,37,6,8,19
2,0,2,32,NaN,NaN,NaN,NaN,NaN,NaN,1,40,6,9,22
3,0,2,33,1.0,499.0,499.0,499.0,499.0,499.0,1,37,6,8,19
4,0,2,38,NaN,NaN,NaN,NaN,NaN,NaN,1,41,6,9,23


In [21]:
matrix.fillna(0, inplace=True)
matrix.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7497000 entries, 0 to 7496999
Data columns (total 14 columns):
date_block_num       int8
shop_id              int8
item_id              int16
item_cnt_month       float16
item_price_open      float16
item_price_close     float16
item_price_min       float16
item_price_mean      float16
item_price_max       float16
city_id              uint8
item_category_id     uint8
category_id          uint8
subcategory_id       uint8
subsubcategory_id    uint8
dtypes: float16(6), int16(1), int8(2), uint8(5)
memory usage: 207.3 MB


In [22]:
def __add_grouped_basics(df, columns, shift_size=None):
    _group_values = df.groupby(columns).agg({'item_cnt_month': ['min', 'mean', 'max']})

    column_base_name = '+'.join(columns)
    new_columns = [
        'item_cnt_month min per {}'.format(column_base_name),
        'item_cnt_month mean per {}'.format(column_base_name),
        'item_cnt_month max per {}'.format(column_base_name),
    ]

    _group_values.columns = new_columns
    _group_values.reset_index(inplace=True)
    
    df = pd.merge(df, _group_values, on=columns, how='left')
    del _group_values
    
    _shift_columns = ['shop_id', 'item_id']

    return df

In [23]:
ts = time.time()
matrix = __add_grouped_basics(matrix, ['date_block_num', 'item_id'])
matrix = __add_grouped_basics(matrix, ['date_block_num', 'shop_id'])
matrix = __add_grouped_basics(matrix, ['date_block_num', 'city_id'])
matrix = __add_grouped_basics(matrix, ['date_block_num', 'category_id'])
time.time() - ts

19.454951286315918

In [24]:
ts = time.time()
matrix = __add_grouped_basics(matrix, ['date_block_num', 'shop_id', 'category_id'])
matrix = __add_grouped_basics(matrix, ['date_block_num', 'shop_id', 'category_id', 'subcategory_id'])
matrix = __add_grouped_basics(matrix, ['date_block_num', 'shop_id', 'city_id'])
time.time() - ts

20.33559560775757

In [25]:
ts = time.time()
matrix = __add_grouped_basics(matrix, ['date_block_num', 'category_id', 'city_id'])
matrix = __add_grouped_basics(matrix, ['date_block_num', 'category_id', 'item_id'])
matrix = __add_grouped_basics(matrix, ['date_block_num', 'category_id', 'subcategory_id'])
matrix = __add_grouped_basics(matrix, ['date_block_num', 'category_id', 'subcategory_id', 'city_id'])
time.time() - ts

32.96780180931091

In [26]:
ts = time.time()
matrix = __add_grouped_basics(matrix, ['date_block_num'])
matrix = __add_grouped_basics(matrix, ['date_block_num', 'shop_id', 'subcategory_id'])

time.time() - ts

17.000518798828125

In [27]:
ts = time.time()
matrix = __add_grouped_basics(matrix, ['date_block_num', 'shop_id', 'subsubcategory_id'])
matrix = __add_grouped_basics(matrix, ['date_block_num', 'item_id', 'city_id'])

time.time() - ts

26.059285402297974

In [28]:
ts = time.time()
matrix = __add_grouped_basics(matrix, ['date_block_num', 'subcategory_id'])
matrix = __add_grouped_basics(matrix, ['date_block_num', 'subsubcategory_id'])

time.time() - ts

19.51977825164795

In [29]:
matrix.to_pickle('data_5.pkl')

In [30]:
ts = time.time()
grouped_columns = ['date_block_num','item_id']
methods = ['first', 'last', 'min', 'mean', 'max']
group = train.groupby(grouped_columns).agg({'item_price': methods})

new_columns = ['item_{}_item_price'.format(method) for method in methods]
group.columns = new_columns
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=grouped_columns, how='left')
matrix[new_columns] = matrix[new_columns].astype(np.float16)

time.time() - ts

7.431118011474609

In [31]:
matrix.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7497000 entries, 0 to 7496999
Data columns (total 70 columns):
date_block_num                                                               int8
shop_id                                                                      int8
item_id                                                                      int16
item_cnt_month                                                               float16
item_price_open                                                              float16
item_price_close                                                             float16
item_price_min                                                               float16
item_price_mean                                                              float16
item_price_max                                                               float16
city_id                                                                      uint8
item_category_id                               

In [32]:
del test
del items
del cats
del shops

In [33]:
train['revenue'] = train['item_price'] *  train['item_cnt_day']


ts = time.time()
grouped_columns = ['date_block_num', 'shop_id']
methods = ['sum']
group = train.groupby(grouped_columns).agg({'revenue': methods})

new_columns = ['date_shop_revenue_{}'.format(method) for method in methods]
group.columns = new_columns
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=grouped_columns, how='left')
matrix[new_columns] = matrix[new_columns].astype(np.float16)

time.time() - ts

10.25257134437561

In [34]:
matrix.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7497000 entries, 0 to 7496999
Data columns (total 71 columns):
date_block_num                                                               int8
shop_id                                                                      int8
item_id                                                                      int16
item_cnt_month                                                               float16
item_price_open                                                              float16
item_price_close                                                             float16
item_price_min                                                               float16
item_price_mean                                                              float16
item_price_max                                                               float16
city_id                                                                      uint8
item_category_id                               

In [35]:
matrix.to_pickle('data_51.pkl')

In [36]:
matrix['month'] = matrix['date_block_num'] % 12

In [37]:
days = pd.Series([31,28,31,30,31,30,31,31,30,31,30,31])
matrix['days'] = matrix['month'].map(days).astype(np.int8)

In [38]:
seasons = pd.Series([0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 0])
matrix['season'] = matrix['month'].map(seasons).astype(np.int8)

In [39]:
matrix.to_pickle('data_5full.pkl')

In [40]:
ts = time.time()
cache = {}
matrix['item_shop_last_sale'] = -1
matrix['item_shop_last_sale'] = matrix['item_shop_last_sale'].astype(np.int8)
for idx, row in matrix.iterrows():    
    key = str(row.item_id)+' '+str(row.shop_id)
    if key not in cache:
        if row.item_cnt_month!=0:
            cache[key] = row.date_block_num
    else:
        last_date_block_num = cache[key]
        matrix.at[idx, 'item_shop_last_sale'] = row.date_block_num - last_date_block_num
        cache[key] = row.date_block_num         
time.time() - ts

698.6369967460632

In [41]:
ts = time.time()
cache = {}
matrix['item_last_sale'] = -1
matrix['item_last_sale'] = matrix['item_last_sale'].astype(np.int8)
for idx, row in matrix.iterrows():    
    key = row.item_id
    if key not in cache:
        if row.item_cnt_month!=0:
            cache[key] = row.date_block_num
    else:
        last_date_block_num = cache[key]
        if row.date_block_num>last_date_block_num:
            matrix.at[idx, 'item_last_sale'] = row.date_block_num - last_date_block_num
            cache[key] = row.date_block_num         
time.time() - ts

437.3858404159546

In [42]:
ts = time.time()
matrix['item_shop_first_sale'] = matrix['date_block_num'] - matrix.groupby(['item_id','shop_id'])['date_block_num'].transform('min')
matrix['item_first_sale'] = matrix['date_block_num'] - matrix.groupby('item_id')['date_block_num'].transform('min')
time.time() - ts

1.2926743030548096

In [43]:
matrix.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7497000 entries, 0 to 7496999
Data columns (total 78 columns):
date_block_num                                                               int8
shop_id                                                                      int8
item_id                                                                      int16
item_cnt_month                                                               float16
item_price_open                                                              float16
item_price_close                                                             float16
item_price_min                                                               float16
item_price_mean                                                              float16
item_price_max                                                               float16
city_id                                                                      uint8
item_category_id                               

In [44]:
ts = time.time()
def fill_na(df):
    for col in df.columns:
        if ('_lag_' in col) & (df[col].isnull().any()):
            if ('item_cnt' in col):
                df[col].fillna(0, inplace=True)         
    return df

matrix = fill_na(matrix)
time.time() - ts

0.660386323928833

In [45]:
matrix.to_pickle('data_lstm_full.pkl')
del matrix
# del cache
del group
del items
del shops
del cats
del train
# leave test for submission
gc.collect();

NameError: name 'items' is not defined